### Input initial parameter

In [352]:
samples=40
n_cov=4

### Create random covariate matrix 

In [353]:
import numpy as np
mat=np.random.random((samples,n_cov))*10

### Check if matrix is invertible

In [354]:
def check_inverse_exist(M):
    if np.linalg.det(M)==0:
        print('Error: Singular Matrix')  
        print('Matrix Rank: '+str(np.linalg.matrix_rank(M)))
        return False
    else:
        return True

### Create hat matrix from covariate matrix

$$ P_{z}=I-Z(Z^{T}Z)^{-1}Z^{T}$$

In [355]:
import numpy as np
import pandas as pd
Z=mat
ZtZ=np.dot(Z.transpose(),Z)
if check_inverse_exist(ZtZ):
    ZtZ_inv=np.linalg.inv(ZtZ)
    Pz=-1*(np.zeros((samples,samples))-np.dot(np.dot(Z,ZtZ_inv),Z.transpose()))
    Pz_visual=pd.DataFrame(Pz)

### Model Formulation
$$minimize~~ -S^{T} P_{z}S + (\sum s_{i})^{2}$$

In [356]:
from pyqubo import Array,Constraint
s=Array.create('s',shape=samples,vartype='SPIN')
S=np.matrix(s)   
H=np.dot(np.dot(s,Pz),S.transpose())
H=H+Constraint(sum(s)**2,label='sum of s squared ==0')
model=H.item().compile()
qubo,offset=model.to_qubo()

bqm=model.to_dimod_bqm()

### QUBO matrix

In [357]:
print(Pz_visual)

          0         1         2         3         4         5         6   \
0   0.166666  0.097210 -0.029755  0.029510  0.072372  0.036957  0.033687   
1   0.097210  0.097352  0.033503 -0.011179  0.036523  0.030764  0.002850   
2  -0.029755  0.033503  0.093429  0.016801 -0.030458 -0.015050 -0.015925   
3   0.029510 -0.011179  0.016801  0.163982  0.008708 -0.031581  0.046458   
4   0.072372  0.036523 -0.030458  0.008708  0.113529  0.094746  0.024636   
5   0.036957  0.030764 -0.015050 -0.031581  0.094746  0.093052  0.006122   
6   0.033687  0.002850 -0.015925  0.046458  0.024636  0.006122  0.020782   
7  -0.016920  0.023197  0.054695  0.001550 -0.000437  0.010140 -0.009845   
8  -0.021107 -0.030875  0.009145  0.085765  0.024707  0.008190  0.023373   
9   0.067930  0.078565  0.011286 -0.068195  0.065169  0.072751 -0.008950   
10 -0.026305  0.022572  0.041051 -0.039970  0.068585  0.086746 -0.013026   
11  0.016268  0.017996 -0.020581 -0.054239  0.090073  0.095727 -0.001448   
12  0.081089

### Optimization 

In [358]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import neal
sampler=EmbeddingComposite(DWaveSampler())
#sampler=neal.SimulatedAnnealingSampler()
response=sampler.sample(bqm,num_reads=100)

### Select one of the result that gives minimum energy as the answer to this problem

In [359]:
from collections import defaultdict    
result=defaultdict(list)
for r in response.data(['sample', 'energy']):
    result[r.energy].append(r.sample)

res=pd.DataFrame()
min_energy=min(result.keys())
for i in list(result[min_energy]):
    temp=pd.DataFrame.from_dict(i,'index').transpose()
    res=res.append(temp)
res=res.drop_duplicates()

In [360]:
res=res.drop_duplicates()
print(res)

   s[0]  s[10]  s[11]  s[12]  s[13]  s[14]  s[15]  s[16]  s[17]  s[18]  ...   \
0     0      0      1      0      0      0      0      1      1      0  ...    

   s[37]  s[38]  s[39]  s[3]  s[4]  s[5]  s[6]  s[7]  s[8]  s[9]  
0      1      1      0     0     1     1     1     0     0     1  

[1 rows x 40 columns]


### Calculate means difference between groups

In [361]:
##########Variance, mean (Compare with randomization technique)##################################
dcov=dict()
group=pd.DataFrame(res.iloc[0,:]).reset_index().drop('index',axis=1).rename({0:'group'},axis=1)
for i in range(n_cov):
    cov=pd.DataFrame(mat[:,i]).rename({0:'bins'},axis=1)
    df=pd.concat([cov, group], axis=1)
    dcov[i]={'mean':df['bins'].mean(),
            'variance':df['bins'].var(),
            'mean_control':df[df['group']==0]['bins'].mean(),
            'variance_control':df[df['group']==0]['bins'].var(),
            'mean_treatment':df[df['group']==1]['bins'].mean(),
            'variance_treatment':df[df['group']==1]['bins'].var()}
#print(pd.DataFrame.from_dict(dcov))
#randomization
group=pd.DataFrame(np.random.randint(0,2,(samples,1))).rename({0:'group'},axis=1)
dcovrand=dict()
for i in range(n_cov):
    cov=pd.DataFrame(mat[:,i]).rename({0:'bins'},axis=1)
    df=pd.concat([cov, group], axis=1)
    dcovrand[i]={'mean':df['bins'].mean(),
            'variance':df['bins'].var(),
            'mean_control':df[df['group']==0]['bins'].mean(),
            'variance_control':df[df['group']==0]['bins'].var(),
            'mean_treatment':df[df['group']==1]['bins'].mean(),
            'variance_treatment':df[df['group']==1]['bins'].var()}
#print(pd.DataFrame.from_dict(dcovrand))


In [362]:
df=pd.DataFrame.from_dict(dcov).transpose()
df['different of means-optimization']=(df['mean_treatment']-df['mean_control']).abs()
df_rand=pd.DataFrame.from_dict(dcovrand).transpose()
df_rand['different of means-before optimization']=(df_rand['mean_treatment']-df_rand['mean_control']).abs()
result = pd.concat([df_rand['different of means-before optimization'],df['different of means-optimization']], axis=1, sort=False)

print(result.mean())

different of means-before optimization    0.803896
different of means-optimization           0.451763
dtype: float64
